In [1]:
import pandas as pd
import pymysql
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# DB에서 장르와 장르코드 가져오는 작업

conn = pymysql.connect(host="localhost",
                       database="firstkino",
                       user="kdigital",
                       password="mysql",
                       charset="utf8")

with conn.cursor() as cursor:
    cursor.execute("SELECT genreCd, genreNm FROM Genre")
    genres = cursor.fetchall()
    cursor.execute("SELECT movieCd, genreCd FROM MovieGenre")
    genre_match = cursor.fetchall()
    cursor.execute("SELECT movieCd, movieNm FROM Movie")
    movies = cursor.fetchall()

conn.commit()

conn.close()

In [3]:
genre_match

(('19158001', 1),
 ('19358008', 1),
 ('19428006', 1),
 ('19468006', 1),
 ('19478012', 1),
 ('19488009', 1),
 ('19508006', 1),
 ('19538009', 1),
 ('19538019', 1),
 ('19548015', 1),
 ('19548016', 1),
 ('19548026', 1),
 ('19558005', 1),
 ('19558008', 1),
 ('19568004', 1),
 ('19568005', 1),
 ('19568016', 1),
 ('19568019', 1),
 ('19598003', 1),
 ('19598012', 1),
 ('19598024', 1),
 ('19608022', 1),
 ('19608108', 1),
 ('19618013', 1),
 ('19618033', 1),
 ('19628015', 1),
 ('19628032', 1),
 ('19638010', 1),
 ('19648083', 1),
 ('19658023', 1),
 ('19658074', 1),
 ('19660068', 1),
 ('19668028', 1),
 ('19678025', 1),
 ('19688015', 1),
 ('19688113', 1),
 ('19698012', 1),
 ('19698020', 1),
 ('19698061', 1),
 ('19698121', 1),
 ('19708023', 1),
 ('19708024', 1),
 ('19708049', 1),
 ('19708128', 1),
 ('19708130', 1),
 ('19718250', 1),
 ('19728127', 1),
 ('19728181', 1),
 ('19728187', 1),
 ('19738127', 1),
 ('19758027', 1),
 ('19768229', 1),
 ('19770077', 1),
 ('19770085', 1),
 ('19770087', 1),
 ('1977009

In [4]:
genre_list = list(genres)
genre_df = pd.DataFrame(genre_list, columns=['genreCd', 'genreNm'])
genre_df

,genreCd,genreNm
0,1,서부극(웨스턴)
1,2,드라마
2,3,SF
3,4,뮤지컬
4,5,공연
5,6,멜로/로맨스
6,7,스릴러
7,8,성인물(에로)
8,9,미스터리
9,10,범죄


In [5]:
match_list = list(genre_match)
match_df = pd.DataFrame(match_list, columns=['movieCd', 'genreCd'])
match_df

,movieCd,genreCd
0,19158001,1
1,19358008,1
2,19428006,1
3,19468006,1
4,19478012,1
...,...,...
73025,20219378,21
73026,20219405,21
73027,20219826,21
73028,20219892,21


In [6]:
movie_list = list(movies)
movie_df = pd.DataFrame(movie_list, columns=['movieCd', 'movieNm'])
movie_df

,movieCd,movieNm
0,19158001,국가의 탄생
1,19188001,개의 일생
2,19190002,여름밤의 기묘한 이야기
3,19190003,재판장
4,19190007,의리적 구토
...,...,...
46913,20219964,생각 끝에서 찾아오는
46914,20219978,볼레로 만들기
46915,20219984,어디에도 없는 시간
46916,20219993,청수탕 사람들


In [7]:
movie_match = pd.merge(movie_df, match_df, on='movieCd')
movie_match

,movieCd,movieNm,genreCd
0,19158001,국가의 탄생,1
1,19158001,국가의 탄생,2
2,19158001,국가의 탄생,14
3,19188001,개의 일생,19
4,19190002,여름밤의 기묘한 이야기,16
...,...,...,...
73025,20219978,볼레로 만들기,11
73026,20219984,어디에도 없는 시간,2
73027,20219993,청수탕 사람들,2
73028,20219993,청수탕 사람들,19


In [8]:
movie_genre = pd.merge(movie_match, genre_df, on='genreCd')
movie_genre

,movieCd,movieNm,genreCd,genreNm
0,19158001,국가의 탄생,1,서부극(웨스턴)
1,19358008,고스트 라이더스,1,서부극(웨스턴)
2,19428006,존 웨인의 약탈자,1,서부극(웨스턴)
3,19468006,하비 걸,1,서부극(웨스턴)
4,19478012,엔젤 앤 배드맨,1,서부극(웨스턴)
...,...,...,...,...
73025,20217740,피델리오,5,공연
73026,20218337,아르코 라이브 연극 깐느로 가는 길,5,공연
73027,20218815,블랙핑크 더 무비,5,공연
73028,20219458,프랑켄슈타인,5,공연


In [9]:
movie_genre_table = movie_genre.pivot_table(index='movieNm', columns='genreNm')
movie_genre_table.fillna(0, inplace=True)
movie_genre_table.head()

genreCd                               \
genreNm                                   SF   가족   공연 공포(호러)    기타 다큐멘터리   
movieNm                                                                     
 나팔꽃과 카세씨                                0.0  0.0  0.0    0.0   0.0   0.0   
 인디애니페스트2016-파노라마2 - 때때로 흐림, 그래도 괜찮아     0.0  0.0  0.0    0.0   0.0   0.0   
 작은 통일                                   0.0  0.0  0.0    0.0   0.0  20.0   
 전쟁 속 우정: Anton                          0.0  0.0  0.0    0.0   0.0   0.0   
 카르미나 부리나                                0.0  0.0  0.0    0.0  11.0   0.0   

                                                                               \
genreNm                               드라마 멜로/로맨스  뮤지컬 미스터리   범죄   사극 서부극(웨스턴)   
movieNm                                                                         
 나팔꽃과 카세씨                             0.0    0.0  0.0  0.0  0.0  0.0      0.0   
 인디애니페스트2016-파노라마2 - 때때로 흐림, 그래도 괜찮아  0.0    0.0  0.0  0.0  0.0  0.0      0.0   
 작은 통일                                0.0    0.0  0.0  0.0  0.0  0.0      0.0   
 전쟁 속 우정: Anton                       2.0    0.0  0.0  0.0  0.0  0.0      0.0   
 카르미나 부리나                             0.0    0.0  0.0  0.0  0.0  0.0      0.0   

                                                                          
genreNm                               스릴러 애니메이션   액션 어드벤처   전쟁  코미디  판타지  
movieNm                                                                   
 나팔꽃과 카세씨                             0.0  12.0  0.0  0.0  0.0  0.0  0.0  
 인디애니페스트2016-파노라마2 - 때때로 흐림, 그래도 괜찮아  0.0  12.0  0.0  0.0  0.0  0.0  0.0  
 작은 통일                                0.0   0.0  0.0  0.0  0.0  0.0  0.0  
 전쟁 속 우정: Anton                       0.0   0.0  0.0  0.0  0.0  0.0  0.0  
 카르미나 부리나                             0.0   0.0  0.0  0.0  0.0  0.0  0.0

In [10]:
movie_similarity_rate = cosine_similarity(movie_genre_table, movie_genre_table)
print(movie_similarity_rate)

[[1. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [11]:
movie_similarity_rate_df = pd.DataFrame(
    data=movie_similarity_rate,
    index=movie_genre_table.index,
    columns=movie_genre_table.index)

In [12]:
movie_similarity_rate_df

movieNm,나팔꽃과 카세씨,"인디애니페스트2016-파노라마2 - 때때로 흐림, 그래도 괜찮아",작은 통일,전쟁 속 우정: Anton,카르미나 부리나,휘파람의 노래: 시벨,"""레드필터가 철회됩니다.""","""있음과 없음"" 전시 영상",# 3_2,#51,...,힘의 한 세기,힙스터: 안개의 덫,힙합 X,힙합 쿠데타,"힙합, 세상은 너의 것","힙합어르신, 라스베이거스에 가다",힛쳐,힛쳐 2,戀情,ＡＶ여배우의 자취방
movieNm,,,,,,,,,,,,,,,,,,,,,
나팔꽃과 카세씨,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
"인디애니페스트2016-파노라마2 - 때때로 흐림, 그래도 괜찮아",1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
작은 통일,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,1.0,1.0,0.000000,0.000000,0.0,0.0
전쟁 속 우정: Anton,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.316228,0.0,1.0,0.0,0.0,0.000000,0.000000,1.0,0.0
카르미나 부리나,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"힙합어르신, 라스베이거스에 가다",0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,1.0,1.0,0.000000,0.000000,0.0,0.0
힛쳐,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,0.695792,0.0,0.0
힛쳐 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.695792,1.000000,0.0,0.0


In [18]:
def recommend_movie(movieNm):
    return movie_similarity_rate_df[movieNm].sort_values(ascending=False)[:51]

In [19]:
recommend_movie("어벤져스")

movieNm
가디언즈 오브 갤럭시 VOL. 2    1.0
미스캡틴 : 섹시 히어로즈        1.0
아이언걸2 - 얼티메이트 웨폰      1.0
전격 Z작전 2000           1.0
스파크맨(Spark Man)       1.0
리볼트 : 프리즈너 오브 워       1.0
홍콩 마스터 어벤져            1.0
저지먼트 데이 : 지구붕괴        1.0
저지드레드 3D              1.0
화랑브이 3총사              1.0
리젼                    1.0
스페이스 워                1.0
스페이스 인커밍              1.0
리터너                   1.0
에볼라 바이러스              1.0
싸이보그 트랙커              1.0
유니버설 솔저:그 두번째 임무      1.0
잭 스나이더의 저스티스 리그       1.0
스포일러                  1.0
싸이보그 2                1.0
리플리컨트                 1.0
하이랜더2                 1.0
태양의전사들                1.0
터미너스: 인류 멸망의 시작       1.0
터미네이터                 1.0
터미네이터 : 미래전쟁의 시작      1.0
윌로우                   1.0
미래전쟁                  1.0
미래경찰 X                1.0
터커와 플린                1.0
압솔롬 탈출                1.0
스파이더맨 3               1.0
타임캅 2                 1.0
전사 카멜레온 2             1.0
포트리스 2                1.0
월드 인베이젼               1.0
킹스 고질라                1.0
에조: 인류 최후의 전쟁         1.0
론울브스